In [1]:
 # YouTube 댓글 다운로드를 위한 라이브러리 설치
!pip install youtube-comment-downloader

# 언어 감지를 위한 라이브러리 설치 (langid: 텍스트의 언어를 감지해주는 라이브러리)
!pip install langid

# Streamlit 설치 (Streamlit은 간단한 웹 앱을 빠르게 만들 수 있게 해주는 라이브러리)
!pip install streamlit

# ngrok을 통해 로컬 서버를 외부에서도 접속 가능하게 해주는 라이브러리 설치 (streamlit 웹앱을 배포할 때 사용 가능)
!pip install pyngrok

# 언어 코드 관련 처리를 위한 라이브러리 설치 (langcodes: ISO 639 언어 코드 관리 및 변환)
!pip install langcodes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941171 sha256=b0c607733b0e08bc1fba6bf373a9069a868becf1925c5223c84cc3b814324681
  Stored in directory: /root/.cache/pip/wheels/32/6a/b6/b7eb43a6ad55b139c15c5daa29f3707659cfa6944d3c696f5b
Successfully built langid
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


기존 버전, 지금 안 씀

In [ ]:
#지금 작성한 코드 블록을 파일로 저장해 주는 기능
%%writefile streamlit_app.py

#____________________________________________________________________________

import streamlit as st  # Streamlit: 웹앱을 쉽게 만들 수 있는 라이브러리
import os  # OS 명령어 실행용
import json  # JSON 데이터 처리
import pandas as pd  # 데이터프레임 처리
import langid  # 언어 감지 라이브러리
import langcodes  # 언어 코드 -> 언어 이름 매핑 라이브러리

#____________________________________________________________________________


# 디자인 CSS 추가: 웹앱의 배경, 폰트, 버튼 등 시각적 스타일 정의

st.markdown("""
<style>
/* 배경 및 글꼴 설정 */
body {
  margin: 0;
  padding: 0;
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  background-color: #ffffff;
  text-align: center;
  color: #333;
}
/* 중앙 로고 및 텍스트 스타일 */
.center-box {
  display: flex;
  flex-direction: column;
  justify-content: center;
  align-items: center;
  margin-top: 50px;
}
.center-box img {
  width: 40px;
  margin-bottom: 20px;
}
.center-box .title {
  font-size: 48px;
  color: #e60000;
  font-weight: bold;
  margin: 10px 0;
}
.center-box .subtitle {
  font-size: 16px;
  color: #888;
  margin-bottom: 30px;
}
/* 입력란과 버튼 스타일 */
.input-style input {
  padding: 12px 20px;
  width: 300px;
  border-radius: 10px;
  border: 1px solid #ccc;
  font-size: 14px;
}
.button-style button {
  padding: 12px 20px;
  background-color: #e60000;
  color: white;
  border: none;
  border-radius: 10px;
  font-weight: bold;
  cursor: pointer;
  font-size: 14px;
}
.button-style button:hover {
  background-color: #cc0000;
}
</style>
""", unsafe_allow_html=True)

# 중앙 로고, 제목, 설명 표시
st.markdown("""
<div class="center-box">
  <img src="https://upload.wikimedia.org/wikipedia/commons/b/b8/YouTube_Logo_2017.svg" alt="YouTube logo" />
  <div class="title">YouniversAI</div>
  <div class="subtitle">Your smart assistant for multilingual YouTube comment insights.</div>
</div>
""", unsafe_allow_html=True)

#____________________________________________________________________________


# YouTube 댓글을 다운로드하여 DataFrame으로 반환하는 함수
def get_comments(url):
    json_file = 'YoutubeComments.json'
    os.system(f'youtube-comment-downloader --url "{url}" --output {json_file}')  # 외부 명령어로 댓글 다운로드 (youtube-comment-downloader 사용)

    # JSON 파일 읽기
    with open(json_file, 'r', encoding='utf-8') as f:
        content = f.read()
        try:
            json_data = json.loads(content)  # 표준 JSON 파싱
        except json.JSONDecodeError:
            # JSON 라인 단위 파싱 (비표준 형식 처리)
            json_data = []
            for line in content.splitlines():
                line = line.strip()
                if not line:
                    continue
                try:
                    json_data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue

    df = pd.DataFrame(json_data)  # pandas DataFrame 생성
    os.remove(json_file)  # 임시 파일 삭제
    return df

#____________________________________________________________________________


# 언어 감지 및 언어 이름 변환 함수
def classify_language(df):
    df['lang'] = df['text'].apply(lambda x: langid.classify(x)[0])  # langid로 언어코드 감지
    df['언어'] = df['lang'].apply(lambda code: langcodes.Language.get(code).display_name() if code else code)  # 언어 코드 -> 언어명 변환
    return df

#____________________________________________________________________________

# Streamlit 웹앱의 메인 함수
def main():
    # URL 입력란과 버튼
    url = st.text_input("Paste YouTube video URL here")
    if st.button("Go"):
        st.write(f"📺 You entered: {url}")

        with st.spinner("🔎댓글을 분류중입니다... 잠시만 기다려주세요"):
            df = get_comments(url)  # 댓글 가져오기
            if df.empty or 'text' not in df.columns:
                st.warning("No comments found or 'text' field missing.")  # 오류 처리
                return
            df = classify_language(df)  # 언어 분류
            st.session_state.df = df  # 세션 상태에 저장 (다른 인터랙션에서도 유지)

    # 댓글 데이터가 있는 경우 출력
    if 'df' in st.session_state:
        df = st.session_state.df
        st.success("댓글 수집 및 언어 분류 완료!")
        st.write("총 댓글 수:", len(df))

        # 언어 선택 멀티셀렉트
        languages = df['lang'].unique().tolist()  # 언어 코드 리스트
        language_options = [langcodes.Language.get(code).display_name() for code in languages]

        selected_langs = st.multiselect("언어를 선택하세요", options=language_options, default=language_options)

        # 선택된 언어에 해당하는 코드 필터링
        selected_lang_codes = [code for code in languages if langcodes.Language.get(code).display_name() in selected_langs]
        filtered_df = df[df['lang'].isin(selected_lang_codes)]

        # 선택된 언어의 댓글 출력
        if not filtered_df.empty:
            st.write(filtered_df[['언어', 'text']])
        else:
            st.warning("선택한 언어에 해당하는 댓글이 없습니다.")

#____________________________________________________________________________

# 프로그램 시작점
if __name__ == "__main__":
    main()


Writing streamlit_app.py


----------------

**디자인**

In [2]:
%%writefile design.py
import streamlit as st

def apply_css():
    st.markdown("""
    <style>
    body {
      margin: 0;
      padding: 0;
      font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
      background-color: #ffffff;
      text-align: center;
      color: #333;
    }
    .center-box {
      display: flex;
      flex-direction: column;
      justify-content: center;
      align-items: center;
      margin-top: 50px;
    }
    .center-box img {
      width: 40px;
      margin-bottom: 20px;
    }
    .center-box .title {
      font-size: 48px;
      color: #e60000;
      font-weight: bold;
      margin: 10px 0;
    }
    .center-box .subtitle {
      font-size: 16px;
      color: #888;
      margin-bottom: 30px;
    }
    .input-style input {
      padding: 12px 20px;
      width: 300px;
      border-radius: 10px;
      border: 1px solid #ccc;
      font-size: 14px;
    }
    .button-style button {
      padding: 12px 20px;
      background-color: #e60000;
      color: white;
      border: none;
      border-radius: 10px;
      font-weight: bold;
      cursor: pointer;
      font-size: 14px;
    }
    .button-style button:hover {
      background-color: #cc0000;
    }
    </style>
    """, unsafe_allow_html=True)

def show_header():
    st.markdown("""
    <div class="center-box">
      <img src="https://upload.wikimedia.org/wikipedia/commons/b/b8/YouTube_Logo_2017.svg" alt="YouTube logo" />
      <div class="title">YouniversAI</div>
      <div class="subtitle">Your smart assistant for multilingual YouTube comment insights.</div>
    </div>
    """, unsafe_allow_html=True)


Writing design.py


**댓글 수집**

In [3]:
%%writefile logic.py
import os
import json
import pandas as pd

def get_comments(url):
    json_file = 'YoutubeComments.json'
    os.system(f'youtube-comment-downloader --url "{url}" --output {json_file}')

    with open(json_file, 'r', encoding='utf-8') as f:
        content = f.read()
        try:
            json_data = json.loads(content)
        except json.JSONDecodeError:
            json_data = []
            for line in content.splitlines():
                line = line.strip()
                if not line:
                    continue
                try:
                    json_data.append(json.loads(line))
                except json.JSONDecodeError:
                    continue

    df = pd.DataFrame(json_data)
    os.remove(json_file)
    return df


Writing logic.py


**댓글 분류**

In [4]:
%%writefile classifier.py
import langid
import langcodes

def classify_language(df):
    df['lang'] = df['text'].apply(lambda x: langid.classify(x)[0])
    df['언어'] = df['lang'].apply(
        lambda code: langcodes.Language.get(code).display_name() if code else code)
    return df

def classify_and_store(df, session_state):
    df = classify_language(df)
    session_state.df = df
    return df


Writing classifier.py


**요약 전용 파일**

In [5]:
%%writefile summarizer.py
import re
import pandas as pd
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT

# 요약 모델 초기화
summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1  # CPU 모드
)

# 키워드 추출 모델 초기화
embed_model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
kw_model = KeyBERT(model=embed_model)

def summarize_by_language(df):
    result = {}
    # summarizer.py 수정
    grouped = df.groupby('lang', dropna=True)


    for lang, group in grouped:
        comments = group['text'].dropna().astype(str).tolist()
        full_text = " ".join(comments)

        if len(full_text) < 50:
            summary = "(댓글이 너무 적어 요약 불가)"
            keywords = ""
        else:
            try:
                summary = summarizer(full_text[:1500], max_length=150, min_length=30, do_sample=False)[0]['summary_text']
            except:
                summary = "(요약 오류 발생)"
            try:
                keywords = ", ".join([kw[0] for kw in kw_model.extract_keywords(full_text)])
            except:
                keywords = ""

        result[lang] = {
            "summary": summary,
            "keywords": keywords
        }

    return result


Writing summarizer.py


**감정 분석 전용 파일**

In [6]:
%%writefile sentiment.py
import langid
import pandas as pd
from transformers import pipeline

# 감정 분석 모델 초기화 (CPU 모드)
sentiment_model = "nlptown/bert-base-multilingual-uncased-sentiment"
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model=sentiment_model,
    tokenizer=sentiment_model,
    truncation=True,
    device=-1
)

# 언어 감지 함수 (간단 버전)
def detect_langid(text):
    try:
        text = str(text).strip()
        if len(text) < 2:
            return 'unknown'
        return langid.classify(text)[0]
    except:
        return 'unknown'

# 감정 분석 함수
def analyze_sentiment(df):
    df['language'] = df['text'].apply(detect_langid)
    dfs_by_lang = {}

    for lang in df['language'].unique():
        df_lang = df[df['language'] == lang].copy()
        if df_lang.empty:
            continue

        comments = df_lang['text'].dropna().astype(str).tolist()
        if not comments:
            continue

        try:
            results = sentiment_pipeline(comments)
        except Exception:
            continue

        df_lang['sentiment'] = [r['label'] for r in results]
        df_lang['score'] = [r['score'] for r in results]
        df_lang['sentiment_score'] = df_lang['sentiment'].apply(lambda l: int(l.split()[0]))
        df_lang['sentiment_kor'] = df_lang['sentiment_score'].apply(
            lambda s: "부정" if s <= 2 else ("중립" if s == 3 else "긍정")
        )

        dfs_by_lang[lang] = df_lang

    if not dfs_by_lang:
        return df.copy()

    return pd.concat(dfs_by_lang.values(), ignore_index=True)


Writing sentiment.py


____________________________________________________________________________

# 메인 실행 파일

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import langcodes
from design import apply_css, show_header
from logic import get_comments
from classifier import classify_and_store

def main():
    apply_css()
    show_header()

    url = st.text_input("Paste YouTube video URL here")
    if st.button("Go"):
        st.write(f"📺 You entered: {url}")
        with st.spinner("🔎댓글을 분류중입니다... 잠시만 기다려주세요"):
            df = get_comments(url)
            if df.empty or 'text' not in df.columns:
                st.warning("No comments found or 'text' field missing.")
                return
            classify_and_store(df, st.session_state)

    if 'df' in st.session_state:
        df = st.session_state.df
        st.success("댓글 수집 및 언어 분류 완료!")
        st.write("총 댓글 수:", len(df))

        languages = df['lang'].unique().tolist()
        language_options = [langcodes.Language.get(code).display_name() for code in languages]

        selected_langs = st.multiselect("언어를 선택하세요", options=language_options, default=language_options)
        selected_lang_codes = [code for code in languages if langcodes.Language.get(code).display_name() in selected_langs]
        filtered_df = df[df['lang'].isin(selected_lang_codes)]

        if not filtered_df.empty:
            st.write(filtered_df[['언어', 'text']])
        else:
            st.warning("선택한 언어에 해당하는 댓글이 없습니다.")

if __name__ == "__main__":
    main()


Overwriting streamlit_app.py


메인 실행 파일 - 요약 전용 파일 적용 버전

In [7]:
!pip install keybert sentence-transformers transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 680.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import langcodes
from design import apply_css, show_header
from logic import get_comments
from classifier import classify_and_store
from summarizer import summarize_by_language  # 요약 모듈 가져오기

def main():
    apply_css()
    show_header()

    url = st.text_input("Paste YouTube video URL here")
    if st.button("Go"):
        st.write(f"📺 You entered: {url}")
        with st.spinner("🔎댓글을 분류중입니다... 잠시만 기다려주세요"):
            df = get_comments(url)
            if df.empty or 'text' not in df.columns:
                st.warning("No comments found or 'text' field missing.")
                return
            classify_and_store(df, st.session_state)

    if 'df' in st.session_state:
        df = st.session_state.df
        st.success("댓글 수집 및 언어 분류 완료!")
        st.write("총 댓글 수:", len(df))

        languages = df['lang'].unique().tolist()
        language_options = [langcodes.Language.get(code).display_name() for code in languages]

        selected_langs = st.multiselect("언어를 선택하세요", options=language_options, default=language_options)
        selected_lang_codes = [code for code in languages if langcodes.Language.get(code).display_name() in selected_langs]
        filtered_df = df[df['lang'].isin(selected_lang_codes)]

        if not filtered_df.empty:
            st.write(filtered_df[['언어', 'text']])

            # ✅ 요약하기 버튼
            if st.button("요약하기"):
                st.session_state.summary = summarize_by_language(filtered_df)

        else:
            st.warning("선택한 언어에 해당하는 댓글이 없습니다.")

        # ✅ 요약 결과 출력 (버튼 클릭 이후에만 표시됨)
        if 'summary' in st.session_state and st.session_state.summary:
            for code in selected_lang_codes:
                summary = st.session_state.summary.get(code)
                lang_name = langcodes.Language.get(code).display_name()

                st.markdown(f"### 💬 {lang_name} 요약")
                if summary:
                    st.markdown(f"**요약:** {summary.get('summary', '(요약 없음)')}")
                    st.markdown(f"**키워드:** {summary.get('keywords', '(키워드 없음)')}")
                else:
                    st.info("요약 결과가 없습니다.")

if __name__ == "__main__":
    main()


Overwriting streamlit_app.py


메인 실행 파일 - 요약 전용 파일 + 감정 분석 전용 파일 적용 버전

위에 '!pip install keybert sentence-transformers transformers --quiet'가 실행되어 있어야 함

In [8]:
%%writefile streamlit_app.py
import streamlit as st
import langcodes
from design import apply_css, show_header
from logic import get_comments
from classifier import classify_and_store
from summarizer import summarize_by_language
from sentiment import analyze_sentiment  # ✅ 감정 분석 추가

def main():
    apply_css()
    show_header()

    url = st.text_input("Paste YouTube video URL here")
    if st.button("Go"):
        st.write(f"\U0001F4FA You entered: {url}")
        with st.spinner("\U0001F50E댓글을 분류중입니다... 잠시만 기다려주세요"):
            df = get_comments(url)
            if df.empty or 'text' not in df.columns:
                st.warning("No comments found or 'text' field missing.")
                return
            classify_and_store(df, st.session_state)

    if 'df' in st.session_state:
        df = st.session_state.df
        st.success("댓글 수집 및 언어 분류 완료!")
        st.write("총 댓글 수:", len(df))

        languages = df['lang'].unique().tolist()
        language_options = [langcodes.Language.get(code).display_name() for code in languages]

        selected_langs = st.multiselect("언어를 선택하세요", options=language_options, default=language_options)
        selected_lang_codes = [code for code in languages if langcodes.Language.get(code).display_name() in selected_langs]
        filtered_df = df[df['lang'].isin(selected_lang_codes)]

        if not filtered_df.empty:
            st.write(filtered_df[['언어', 'text']])

            # ✅ 감정 분석 버튼 추가
            if st.button("감정 분석하기"):
                with st.spinner("\U0001F914 감정을 분석 중입니다..."):
                    df_with_sentiment = analyze_sentiment(filtered_df)
                    st.session_state.df = df_with_sentiment  # 업데이트
                    st.success("감정 분석 완료!")
                    st.dataframe(df_with_sentiment[['언어', 'text', 'sentiment_kor']])

            # ✅ 요약 버튼 추가
            if st.button("요약하기"):
                with st.spinner("\u23F3 언어별 요약 중입니다..."):
                    st.session_state.summary = summarize_by_language(filtered_df)

                for lang_code in selected_lang_codes:
                    summary = st.session_state.summary.get(lang_code)
                    lang_label = langcodes.Language.get(lang_code).display_name()
                    st.markdown(f"### 💬 {lang_label} 요약")
                    if summary:
                        st.markdown(f"**요약:** {summary.get('summary', '(요약 없음)')}")
                        st.markdown(f"**키워드:** {summary.get('keywords', '(키워드 없음)')}")
                    else:
                        st.info("요약 결과가 없습니다.")

        else:
            st.warning("선택한 언어에 해당하는 댓글이 없습니다.")

if __name__ == "__main__":
    main()


Writing streamlit_app.py


________________________________________________________________________

In [ ]:
from pyngrok import ngrok  # pyngrok 라이브러리를 임포트하여 ngrok과 연동


# ngrok 토큰 등록 (여기에 본인 ngrok 계정의 인증 토큰을 입력해야 함)
ngrok.set_auth_token("")



# 연결할 포트를 지정하고, ngrok으로 해당 포트를 외부에서 접속 가능하게 연결
port = 8501  # Streamlit의 기본 포트가 8501
public_url = ngrok.connect(port)  # 포트 8501을 외부 접속용으로 공개
print(f"Public URL: {public_url}")  # 생성된 public URL을 출력



# Streamlit 앱 실행 (streamlit_app.py 파일을 실행하고, 지정된 포트로 서버를 시작)
# &는 백그라운드 실행 (Colab 같은 환경에서는 필요)
!streamlit run streamlit_app.py --server.port {port} &
